In [ ]:
import json
import urllib.request
import matplotlib.pyplot as plt
import re
import os
import networkx as nx
from networkx.algorithms.community import louvain_communities
from concurrent.futures import ThreadPoolExecutor
from urllib.request import urlopen
import matplotlib.pyplot as plt
import networkx as nx
from bs4 import BeautifulSoup
import os
import requests
from urllib.parse import urljoin

In [ ]:
raw_response=urlopen("https://onepiece.fandom.com/wiki/List_of_Canon_Characters")
response=raw_response.read()
soup=BeautifulSoup(response,"html.parser")
table = soup.find("table", class_="fandom-table sortable")

if table:
    tbody = table.find("tbody")
    rows = tbody.find_all("tr")
    names={}
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 1: 
            name_link = cols[1].find("a")
            if name_link:
                names[name_link.text] = name_link.get("href")

In [ ]:


base_url = "https://onepiece.fandom.com"

output_folder = "C:\\Users\\17675\\Desktop\\02450\\onepiece"
os.makedirs(output_folder, exist_ok=True)
timeout_links = {}
for name, relative_link in names.items():
    full_url = urljoin(base_url, relative_link)

    try:
        response = requests.get(full_url,timeout=2)
        response.raise_for_status()
        file_path = os.path.join(output_folder, f"{name}.txt")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(response.text)

        print(f"Saved {name} page to {file_path}")

    except requests.exceptions.Timeout:
        timeout_links[name] = full_url

    except requests.exceptions.RequestException as e:
        print(f"Failed to download {name}: {e}")

In [ ]:
# Function to read all file contents into a dictionary
def load_files_to_memory(file_paths):
    content_dict = {}
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            content_dict[os.path.splitext(os.path.basename(file_path))[0]] = file.read()
    return content_dict

# Function to process one file and find relationships
def process_file(person_name, file_content, all_names):
    edges = []
    for other_person in all_names:
        if other_person != person_name and other_person in file_content:
            edges.append((person_name, other_person))
    return edges

# Get file paths and names
file_paths = [os.path.join('onepiece', f) for f in os.listdir(os.path.join('onepiece'))]
file_names = [os.path.splitext(os.path.basename(f))[0] for f in file_paths]

# Load all file contents into memory
file_contents = load_files_to_memory(file_paths)

# Initialize a graph
graph = nx.Graph()
graph.add_nodes_from(file_names)  # Add all nodes

# Use multithreading to find edges
edges = []
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(process_file, person_name, content, file_names)
        for person_name, content in file_contents.items()
    ]
    for future in futures:
        edges.extend(future.result())

# Add edges to the graph
graph.add_edges_from(edges)

# Summary of the graph
print("Number of nodes:", graph.number_of_nodes())
print("Number of edges:", graph.number_of_edges())



Number of nodes: 1486
Number of edges: 289633


In [12]:

# Perform Louvain community detection
communities = louvain_communities(graph)

# Assign community labels as attributes to nodes
community_mapping = {}
for community_id, community_nodes in enumerate(communities):
    for node in community_nodes:
        community_mapping[node] = community_id

# Add community attribute to each node
nx.set_node_attributes(graph, community_mapping, "community")

# Export the graph with community data
output_path = "onepiece_person_relationships.gexf"
nx.write_gexf(graph, output_path)

# Print summary of detected communities
print("Number of communities:", len(communities))
print("Community sizes:", [len(c) for c in communities])


Number of communities: 6
Community sizes: [257, 246, 167, 184, 246, 386]


In [ ]:
# import os
# import networkx as nx
# from bs4 import BeautifulSoup
# from concurrent.futures import ThreadPoolExecutor

# # Function to read and parse file content
# def extract_titles_from_file(file_path):
#     with open(file_path, 'r', encoding='utf-8') as file:
#         content = file.read()
#         soup = BeautifulSoup(content, 'html.parser')
#         return [tag['title'] for tag in soup.find_all('a', title=True)]

# # Function to find relationships based on extracted titles
# def process_file_with_titles(person_name, file_titles, all_titles):
#     edges = []
#     for other_person in all_titles:
#         if other_person != person_name and other_person in file_titles:
#             edges.append((person_name, other_person))
#     return edges

# # Get file paths and names
# file_paths = [os.path.join('onepiece', f) for f in os.listdir(os.path.join('onepiece'))]
# file_names = [os.path.splitext(os.path.basename(f))[0] for f in file_paths]

# # Extract titles from all files
# titles_dict = {}
# with ThreadPoolExecutor() as executor:
#     futures = {executor.submit(extract_titles_from_file, file_path): file_name for file_path, file_name in zip(file_paths, file_names)}
#     for future in futures:
#         file_name = futures[future]
#         titles_dict[file_name] = future.result()

# # Initialize a graph
# graph = nx.Graph()
# graph.add_nodes_from(file_names)  # Add all nodes

# # Use multithreading to find edges
# edges = []
# with ThreadPoolExecutor() as executor:
#     futures = [
#         executor.submit(process_file_with_titles, person_name, titles, set(titles_dict.keys()))
#         for person_name, titles in titles_dict.items()
#     ]
#     for future in futures:
#         edges.extend(future.result())

# # Add edges to the graph
# graph.add_edges_from(edges)

# # Summary of the graph
# print("Number of nodes:", graph.number_of_nodes())
# print("Number of edges:", graph.number_of_edges())

# # Export the graph for visualization
# nx.write_gexf(graph, "person_relationships_onepiece.gexf")


['Epithet', 'Monkey D. Luffy', 'Roronoa Zoro', 'Swordsman', 'Nami', 'Navigator', 'Usopp', 'Sniper', 'Sanji', 'Cook', 'Tony Tony Chopper', 'Doctor', 'Nico Robin', 'Archaeologist', 'Franky', 'Shipwright', 'Brook', 'Musician', 'Jinbe', 'Helmsman', 'Nefertari Vivi']
